In [7]:
pip install streamlit


     ---------------------------------------- 0.0/44.3 kB ? eta -:--:--
     -------------------------- ----------- 30.7/44.3 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 44.3/44.3 kB 724.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.0 MB ? eta -:--:--
    --------------------------------------- 0.1/9.0 MB 2.1 MB/s eta 0:00:05
   - -------------------------------------- 0.2/9.0 MB 2.0 MB/s eta 0:00:05
   - -------------------------------------- 0.4/9.0 MB 2.2 MB/s eta 0:00:04
   - -------------------------------------- 0.4/9.0 MB 2.0 MB/s eta 0:00:05
   -- ------------------------------------- 0.5/9.0 MB 2.2 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/9.0 MB 2.2 MB/s eta 0:00:04
   --- ------------------------------------ 0.8/9.0 MB 2.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/9.0 MB 2.5 MB/s eta 0:00:04
   ---- -------------

DEPRECATION: Loading egg at d:\python\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pickle
import streamlit as st


## Parsing

Como primer paso lo que se hará será dar orden al archivo base dónde se encuentran los datos. Actualmente esos datos son útiles pero no los podemos manipular para limpiarlos y pre-procesarlos para alimentar el algoritmo, por lo que este paso es fundamental en la creación del df.


In [2]:
def parse_log (file_path):
    """ 
    Función de Ingeniería: Lee un archivo de bitacora crudo y lo estructura.
    Entrada: Ruta del archivo .txt
    Salida : DataFrame de Pandas 
    """

    #Definir la lista donde guardaremos los bloques procesados
    data_list = []
    #Crear un diccionario donde se gurdaran los mensajes
    current_record = {}

    with open(file_path, 'r',encoding='utf-8') as f:
        for line in f:
            line = line.strip() # Se quitan los espacios vacíos al inicio/final

            #Si la línea está vacío significa que terminó el bloque
            if not line:
                if current_record: #Solo se guardan cuando hay datos
                    data_list.append(current_record)
                    current_record = {}
                continue
            if ':' in line:
                key, value = line.split(':',1)
                key = key.strip()
                value = value.strip()
                #Con esta información se llena el diccionario
                current_record[key] = value
        # Si el archivo termina y quedó un último récrod sin guardar
        if current_record:
            data_list.append(current_record)

    df = pd.DataFrame(data_list)
    return df

file_name = 'conversacion.txt'

try:
    df_mess = parse_log(file_name)
    df_mess['Fecha'] = '2025-11-27'
    print(f'Ejecución correcta: {df_mess.shape}')
    print(df_mess.head(5))
except FileNotFoundError:
    print('No se encontró el archivo.')
except Exception as e:
    print(f'Error {e}')

Ejecución correcta: (261, 9)
                 LE     HO                        UD  \
0         Entrada 1  06:15     Camión-23 p/JK-8821-C   
1  Garita Principal  06:22  STR 087 placas VM-43-771   
2        Campamento  06:30   Pipa CR-445 p/NM-7723-A   
3         Entrada 4  06:35       DSM-4521 p/LK-92-3B   
4            Planta  06:40             Getman NM-125   

                           EV     ES  \
0              Roberto Méndez    JKF   
1     inspección de seguridad    2-S   
2             Laura Fernández    JKF   
3  Inicio de labores operador  CIDSA   
4     inspección de seguridad    OKK   

                                                  DA                QN  \
0                                  Inicio de Labores    Mario Castillo   
1        Inicio de Labores personal de mantenimiento  Carlos Domínguez   
2  Abastecimiento de Diesel: 450 lts H.I 892341 H...     Pedro Sánchez   
3                                  Inicio de Labores    Javier Morales   
4                      

El siguiente paso es empezar a segmentar los mensajes por información que hay dentro de ellos. Para esto primero debemos limpiar el texto y después con base en palabras clave hacer la categorización

In [3]:
#Definimos los diccionarios, con las palabras clave y su categoría
Keywords = {
    'Mantenimiento':['falla','aceite','fuga','ruido','motor','frenos','combustible','neumatico','ponchado','luces','balatas','hidraulico','taller','almacen','repuesto','reparacion','rota'],
    'Seguridad':['desprendimiento','temperatura','obstruida','lluvia','neblina','hielo','resbaloso','casco','epp','riesgo','derrumbe','polvo','accidente','cuidado','alerta','situacion','peligro'],
    'Operacion':['inicio','fin','turno','comedor','llena','viaje','ok','enterado','copiado'],
    'LOGISTICA': ['diesel', 'gasolina', 'combustible', 'agua', 'almacen', 'stock', 'pedido', 'llegada', 'transporte', 'bus', 'camioneta', 'chofer', 'material', 'reactivo', 'explosivo', 'proveedor'
    ]
}

def clean_text(text):
    """
    Normaliza el texto: Minúsculas y quita caracteres especiales
    """
    if pd.isna(text): return ""
    text = str(text).lower()
    text = re.sub(r'[^\w\s]','', text)
    return text

def label_message(text):
    """
    Función que asigna una categoría con base en una palabra clave
    """

    text = str(text).lower()
    #Se buscan coincidencia con el diccionario:
    for category, keywords in Keywords.items():
        for word in keywords:
            if word in text:
                return category
    #Si no encuentra palabras clave de Mantenimiente o Seguridad se asume operación
    return 'Operacion'

#Se hace una columna nueva para no borrar la original
df_mess['Texto_limpio'] = df_mess['DA'].apply(clean_text)
#Se crea la columna que será el target del df
df_mess['Categoría'] = df_mess['DA'].apply(label_message)

conteo = df_mess['Categoría'].value_counts()
print(conteo)
print(df_mess[['DA','Categoría']].sample(5))


Categoría
Operacion        148
Mantenimiento     71
Seguridad         25
LOGISTICA         17
Name: count, dtype: int64
                                                    DA      Categoría
18               Unidad operativa transportando equipo      Operacion
169  Carter de aceite con fisura fuga importante de...  Mantenimiento
222  Alarma de evacuación activada por error person...      Operacion
147     Motor de arranque no gira selenoide defectuoso  Mantenimiento
253  Operador sin línea de vida en trabajo en altur...      Operacion


In [6]:
df_mess = df_mess.dropna(subset=['Texto_limpio'])
X = df_mess['Texto_limpio']
y = df_mess['Categoría']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Datos de entrenamiento: {len(X_train)} mensajes')
print(f'Datos de prueba: {len(X_test)} mensajes')

#Se crea el convertidor de texto a números
vectorizer = CountVectorizer()

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

#Entrenamiento del modelos
# Usamos Naive Bayes (MultinomialNB), es el estándar de oro para texto simple y rápido.
model = MultinomialNB()
model.fit(X_train_vec, y_train)
#Evaluación
predictions = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, predictions)
print(f'Precisión del Modelo (Accuracy): {accuracy:.2f}%')
print(classification_report(y_test, predictions))

#Guardar el modelo como el vectorizador para usarlos en la Web
with open('mining_model.pkl','wb') as f:
    pickle.dump(model,f)

with open('vectorizer.pkl','wb') as f:
    pickle.dump(vectorizer,f)

print('Modelo y Vectorizador guardados como archivos .pkl')


Datos de entrenamiento: 208 mensajes
Datos de prueba: 53 mensajes
Precisión del Modelo (Accuracy): 0.83%
               precision    recall  f1-score   support

    LOGISTICA       1.00      1.00      1.00         5
Mantenimiento       0.74      0.88      0.80        16
    Operacion       0.86      0.86      0.86        28
    Seguridad       1.00      0.25      0.40         4

     accuracy                           0.83        53
    macro avg       0.90      0.75      0.76        53
 weighted avg       0.85      0.83      0.82        53

Modelo y Vectorizador guardados como archivos .pkl


https://gemini.google.com/gem/429a49f35c66/a75c32d6e89ae9a5?android-min-version=301356232&ios-min-version=322.0&is_sa=1&campaign_id=test_autosubmit&pt=9008&mt=8